In [33]:
import pandas as pd
import tkinter as tk
import csv
from tkinter import filedialog
import numpy as np

root = tk.Tk()
f={}
xls=None
number_of_points=None
height = None
def import_design_data():
    root.filename=filedialog.askopenfilename(filetypes = [('xlsx files' , '*.xlsx')])
    df1 = pd.read_excel(root.filename)
    f["design_data"]=df1

def import_pre_post_data():
    root.filename1=filedialog.askopenfilename(filetypes = [('xlsx files' , '*.xlsx')])
    global xls
    xls = pd.ExcelFile('prepostdata.xlsx')
    df2 = pd.read_excel(root.filename1)
    f["prepostdata"]=df2


def points(p):
    global number_of_points
    number_of_points = int(p)
    
def HEIGHT(h):
    global height
    height = float(h)

def write_to_csv():
    dirname = filedialog.askdirectory()
    
    design_data = f["design_data"]
    d_all={}
    for i in xls.sheet_names:
        df = pd.read_excel('prepostdata.xlsx',sheet_name=i)
        chainage = float(i)
        index=design_data[design_data["CH"]==chainage].index[0]
        d_all[chainage]={"chaingae" : chainage,
                 "base" : design_data.loc[index,'CL_RL'],
                 "OFFSET" : design_data.loc[index,'OFFSET'],
                  "df" : df}
    



    ## DONT TOUCH THIS BLOCK
    for key in d_all:    
        
        a=d_all[key]["df"].copy()

        a.rename(columns={ a.columns[2]: "postwork" }, inplace = True)
        a.rename(columns={ a.columns[1]: "prework" }, inplace = True)
        a.rename(columns={ a.columns[0]: "Distance" }, inplace = True)

        offset=d_all[key]["OFFSET"]
        if offset not in a["Distance"].unique():
            positive_row = {"Distance":offset}
            negative_row = {"Distance":-offset}

            a= a.append(positive_row,ignore_index=True)
            a= a.append(negative_row,ignore_index=True)

            a= a.sort_values(by=['Distance']).reset_index(drop=True)

            a= a.interpolate(method='linear', axis=0).ffill().bfill()

        p_index=int(a[a['Distance']==offset].index.values)
        n_index=int(a[a['Distance']==-offset].index.values)
        l_index=a.shape[0]-1

        ## PREPARING DESIGN SECTION 
        a.insert(loc=3, column='design_section', value=d_all[key]["base"])

        j=n_index-1
        while j>=0:
            a.loc[j,"design_section"]= a["design_section"][n_index] + (1/3)*abs(a["Distance"][n_index]-a["Distance"][j])
            j=j-1

        k=p_index+1
        while k<=l_index:
            a.loc[k,"design_section"] = a["design_section"][n_index] + (1/3)*abs(a["Distance"][p_index]-a["Distance"][k])
            k=k+1

        a["workable_post"]=a["postwork"]
        a.loc[a["workable_post"]>a["prework"], 'workable_post'] = a[a["workable_post"]>a["prework"]]["prework"]
        s = pd.Series(np.sign(a['prework']-a['workable_post']))
        a.insert(loc=5, column='workable_post_sign', value=s)
        a["check"]=a["prework"]-a["workable_post"]
        a["this"] = np.nan

        q=list(range(1,number_of_points+1))
        for item in q:
            sett = [0,*np.ones(item),0]
            for i in range(1,a.shape[0]-item):
                li = list(a.loc[i-1:i+item,"workable_post_sign"])
                if li==sett and (np.array(a.loc[i:i+item-1,"check"])<height).sum()==item:
                    a.loc[i:i+item-1,"this"]="Y"
            if (np.array(a.iloc[0:item,6])<height).sum()==item and (np.array(a.iloc[0:item,5])).sum()==item and a.iloc[item,5]==0:
                a.loc[0:item-1,"this"]="Y"
            if (np.array(a.iloc[-item:,6])<height).sum()==item and (np.array(a.iloc[-item:,5])).sum()==item and a.iloc[-1-item,5]==0:
                a.iloc[-item:,7]="Y"

        if "Y" in a["this"].values:
            a.loc[a["this"]=="Y", 'workable_post'] = a.loc[a["this"]=="Y" ,"prework"]

        a.drop(["workable_post_sign","check","this"], axis=1,inplace=True)

        a_right = a.iloc[:,[0,1,3]]
        a_right.head()

        a_right["sign"]=np.sign(a_right["prework"]-a_right["design_section"])


        rows_to_add4=[]
        for i in range(0,a_right.shape[0]-1):
            if abs(a_right.loc[i,"sign"]-a_right.loc[i+1,"sign"])==2:
                di1 = x1 = x3 = a_right.iloc[i,0]
                di2 = x4 = x2 = a_right.iloc[i+1,0]
                pr1 = y1 = a_right.iloc[i,1]
                pr2 = y2 = a_right.iloc[i+1,1]
                de1 = y3 = a_right.iloc[i,2]
                de2 = y4 = a_right.iloc[i+1,2]
                t = ((x1-x3)*(y3-y4)-(y1-y3)*(x3-x4)) / ((x1-x2)*(y3-y4)-(y1-y2)*(x3-x4))
                x = x1 + t * (x2-x1)
                y = y1 + t * (y2-y1)

                di = x
                pr = de =y
                data = {'Distance':di,'prework':pr,'design_section':de,'sign': 0}
                rows_to_add4.append(data)
        if rows_to_add4:
            a_right = a_right.append(rows_to_add4,ignore_index=True,sort=False)
            a_right = a_right.sort_values(by=['Distance']).reset_index(drop=True)


        a_right.loc[a_right["sign"]==-1, 'design_section'] = a_right[a_right["sign"]==-1]["prework"]

        a_right.drop("sign",axis=1,inplace=True)
        a_right["sign"]=np.sign(a_right["prework"]-a_right["design_section"])

        ## start and end point of right part
        if a_right.loc[0,"sign"]==1:
            a_right_start = 0
        elif np.all(a_right["sign"]==0):
            a_right_start = 0
        else:
            i=0
            while a_right.loc[i,"sign"]<=0:
                i=i+1
            a_right_start = i-1

        if a_right.iloc[-1,3]==1:
            a_right_end = a_right.shape[0]-1
        elif np.all(a_right["sign"]==0):
            a_right_end = a.shape[0]-1
        else:
            i=-1
            while a_right.iloc[i,3]==0:
                i=i-1
            a_right_end = a_right.shape[0]+i+1

        a_right =  a_right.iloc[a_right_start : a_right_end+1,:]
        a_right = a_right.reset_index(drop=True)
        a_right.drop("sign",axis=1,inplace=True)


        a.drop("postwork",axis=1,inplace=True)
        a.insert(loc=2, column='postwork', value=a["workable_post"])
        a.drop('workable_post', axis=1, inplace=True)


        s1 = pd.Series(np.sign(a['prework']-a['postwork']))
        a.insert(loc=4, column='pre_post_sign', value=s1)


        if a.loc[0,"pre_post_sign"]==1:
            a_left_start = 0
        elif np.all(a["pre_post_sign"]==0):
            a_left_start = 0
        else:
            i=0
            while a.loc[i,"pre_post_sign"]==0:
                i=i+1
            a_left_start = i-1

        if a.iloc[-1,4]==1:
            a_left_end = a.shape[0]-1
        elif np.all(a["pre_post_sign"]==0):
            a_left_end = a.shape[0]-1
        else:
            i=-1
            while a.iloc[i,4]==0:
                i=i-1
            a_left_end = a.shape[0]+i+1

        a_left =  a.iloc[a_left_start : a_left_end+1,:]
        a_left = a_left.reset_index(drop=True)

        a_left.drop("pre_post_sign",axis=1,inplace=True)





        a_left["design_pre_sign"] = np.sign(a_left["prework"] - a_left["design_section"])

        rows_to_add1=[]
        for i in range(0,a_left.shape[0]-1):
            if abs(a_left.loc[i,"design_pre_sign"]-a_left.loc[i+1,"design_pre_sign"])==2:
                #print(i)
                di1 = x1 = x3 = a_left.iloc[i,0]
                di2 = x4 = x2 = a_left.iloc[i+1,0]
                pr1 = y1 = a_left.iloc[i,1]
                pr2 = y2 = a_left.iloc[i+1,1]
                de1 = y3 = a_left.iloc[i,3]
                de2 = y4 = a_left.iloc[i+1,3]
                #print(di1)
                t = ((x1-x3)*(y3-y4)-(y1-y3)*(x3-x4)) / ((x1-x2)*(y3-y4)-(y1-y2)*(x3-x4))
                x = x1 + t * (x2-x1)
                y = y1 + t * (y2-y1)

                di = x
                pr = de =y
                post = a_left.iloc[i+1,2] + (di-di2)*((a_left.iloc[i+1,2] -a_left.iloc[i,2] )/(di2-di1))
                #print(pre)
                data = {'Distance':di,'postwork':post,'prework':pr,'design_section':de,'design_pre_sign': 0 }
                rows_to_add1.append(data)
        if rows_to_add1:
            a_left = a_left.append(rows_to_add1,ignore_index=True,sort=False)
            a_left = a_left.sort_values(by=['Distance']).reset_index(drop=True)


        a_left["workable_design"]=a_left["design_section"]

        a_left.loc[a_left["design_pre_sign"]==-1, 'workable_design'] = a_left[a_left["design_pre_sign"]==-1]["prework"]

        a_left.drop("design_pre_sign",axis=1,inplace=True)

        a_left=a_left.round(2)

        a_left["design_post_sign"] = np.sign(a_left["postwork"] - a_left["workable_design"])


        rows_to_add2=[]
        for i in range(0,a_left.shape[0]-1):
            if abs(a_left.loc[i,"design_post_sign"]-a_left.loc[i+1,"design_post_sign"])==2:
                di1 = x1 = x3 = a_left.iloc[i,0]
                di2 = x4 = x2 = a_left.iloc[i+1,0]
                po1 = y1 = a_left.iloc[i,2]
                po2 = y2 = a_left.iloc[i+1,2]
                de1 = y3 = a_left.iloc[i,4]
                de2 = y4 = a_left.iloc[i+1,4]
                t = ((x1-x3)*(y3-y4)-(y1-y3)*(x3-x4)) / ((x1-x2)*(y3-y4)-(y1-y2)*(x3-x4))
                x = x1 + t * (x2-x1)
                y = y1 + t * (y2-y1)

                di = x
                po = de =y
                pre_value = a_left.iloc[i+1,1] + (di-di2)*((a_left.iloc[i+1,1] -a_left.iloc[i,1] )/(di2-di1))
                design_value = a_left.iloc[i+1,3] + (di-di2)*((a_left.iloc[i+1,3] -a_left.iloc[i,3] )/(di2-di1))
                data = {'Distance':di,'postwork':po,'prework':pre_value,'design_section':design_value,'workable_design': de , 'design_post_sign': 0}
                rows_to_add2.append(data)
        if rows_to_add2:
            a_left = a_left.append(rows_to_add2,ignore_index=True,sort=False)
            a_left = a_left.sort_values(by=['Distance']).reset_index(drop=True)


        a_left["Final"]=a_left["postwork"]

        a_left.loc[a_left["design_post_sign"]==-1, 'Final'] = a_left[a_left["design_post_sign"]==-1]["workable_design"]

        a_left.drop(["workable_design","design_post_sign"],axis=1, inplace=True)

        a_left.drop("design_section",axis=1, inplace=True)

        zero_level1 = a_left[['prework','postwork',"Final"]].min(axis=1).min()
        zero_level = np.minimum(0,zero_level1)

        for col in range(1,4):

            ss=a_left.columns[col]
            a_left['{}_area'.format(ss)]=a_left.index.map(lambda x: (a_left.iloc[x,0]-a_left.iloc[x-1,0])*0.5*(a_left.iloc[x,col]+a_left.iloc[x-1,col]-(2*zero_level)) if a_left.iloc[x,0]!=a_left.iloc[0,0] else np.nan)

        a_left = a_left.round(2)

        zero_rows=[]
        if 0 in a_left["Distance"].values:
            a_left_zero_row = a_left[a_left['Distance'] == 0].index[0] 
            zero_rows.append(a_left_zero_row)

        a_left = a_left[['Distance', 'prework','prework_area','Distance', 'postwork','postwork_area','Distance', 'Final','Final_area']]

        for col in range(1,3):
            jj=a_right.columns[col]
            a_right['{}_area'.format(jj)]=a_right.index.map(lambda x: (a_right.iloc[x,0]-a_right.iloc[x-1,0])*0.5*(a_right.iloc[x,col]+a_right.iloc[x-1,col]-(2*zero_level)) if a_right.iloc[x,0]!=a_right.iloc[0,0] else np.nan)

        a_right=a_right.round(2)

        if 0 in a_right["Distance"].values:
            a_right_zero_row = a_right[a_right['Distance'] == 0].index[0] 
            zero_rows.append(a_right_zero_row)

        a_right=a_right[['Distance', 'prework','prework_area','Distance', 'design_section','design_section_area']]


        if len(zero_rows)<2:
            if a_left.shape[0]==a_right.shape[0]:
                pass
            elif a_left.shape[0]>a_right.shape[0]:
                add = a_left.shape[0] - a_right.shape[0]
                a_right=a_right.reindex(a_right.index.tolist() + list(range(a_right.shape[0], a_right.shape[0]+add)))
                a_right = a_right.sort_index().reset_index(drop=True)
            else:
                add = a_right.shape[0] - a_left.shape[0]
                a_left=a_left.reindex(a_left.index.tolist() + list(range(a_left.shape[0], a_left.shape[0]+add)))
                a_left = a_left.sort_index().reset_index(drop=True)



        if len(zero_rows)==2:
            if a_left_zero_row==a_right_zero_row:
                pass
            elif a_left_zero_row>a_right_zero_row:
                add = a_left_zero_row-a_right_zero_row
                a_right=a_right.reindex(a_right.index.tolist() + list(range(-add, 0)))
            else:
                add = a_right_zero_row-a_left_zero_row
                a_left=a_left.reindex(a_left.index.tolist() + list(range(-add, 0)))


            if a_left.shape[0] == a_right.shape[0]:
                pass
            elif a_left.shape[0]>a_right.shape[0]:
                add = a_left.shape[0] - a_right.shape[0]
                a_right=a_right.reindex(a_right.index.tolist() + list(range(a_right.shape[0], a_right.shape[0]+add)))
            else:
                add = a_right.shape[0] - a_left.shape[0]
                a_left=a_left.reindex(a_left.index.tolist() + list(range(a_left.shape[0], a_left.shape[0]+add)))

            a_left = a_left.sort_index().reset_index(drop=True)
            a_right = a_right.sort_index().reset_index(drop=True)


        df = pd.concat([a_left, a_right], axis=1)


        df.rename(columns={ df.columns[0]: "Distance", df.columns[1]: "Levels",df.columns[2]: "Area",
                          df.columns[3]: "Distance", df.columns[4]: "Levels",df.columns[5]: "Area",
                          df.columns[6]: "Distance", df.columns[7]: "Levels",df.columns[8]: "Area",
                          df.columns[9]: "Distance", df.columns[10]: "Levels",df.columns[11]: "Area",
                          df.columns[12]: "Distance", df.columns[13]: "Levels",df.columns[14]: "Area"}, 
                          inplace = True)

        df=pd.concat([df.iloc[:,[0,1,2]], 
                  df.iloc[:,[3,4,5]],
                  df.iloc[:,[6,7,8]],
                  df.iloc[:,[9,10,11]],
                  df.iloc[:,[12,13,14]]],  
                  axis=1,  
                  keys=('Pre-dredge Section','Post-dredged Section',"Dredged Section within Design",'Pre dredge Section','Design Section'))

        df=pd.concat([df.iloc[:,0:9], 
                  df.iloc[:,9:]],  
                  axis=1,  
                  keys=('As per Dredged Section','As per Design Section'))

        df=df.append(pd.Series(), ignore_index=True)
        for u in [2,5,8,11,14]:
            df.iloc[-1,u]=np.sum(df.iloc[1:,u])

        df.iloc[-1,0]="Pre-Dredge Area"
        df.iloc[-1,3]="Post-Dredged Area"
        df.iloc[-1,6]="Dredged Area within Design"
        df.iloc[-1,9]="Pre- Dredge Area"
        df.iloc[-1,12]="Design Dredge Area"

        df=df.append(pd.Series(), ignore_index=True)

        df.iloc[-1,5]=df.iloc[-2,2]-df.iloc[-2,5]
        df.iloc[-1,8]=df.iloc[-2,2]-df.iloc[-2,8]
        df.iloc[-1,14]=df.iloc[-2,11]-df.iloc[-2,14]
        df.iloc[-1,3]="Net Area"

        df = df.round(2)
        d_all[key]["final_df"] = df
        
    frames={}
    for key in d_all:
        frames["{}".format(key)]=d_all[key]["final_df"]

    writer = pd.ExcelWriter('{}\gui.xlsx'.format(dirname), engine='xlsxwriter')
    
    for sheet, frame in  frames.items():
        frame.to_excel(writer, sheet_name=sheet)
    writer.save() 
    
    
canvas = tk.Canvas(root, height = 500, width = 400)
canvas.pack()


frame = tk.Frame(root, bg="#ffbf00", bd=5)
frame.place(relx=.5, rely=.1, relheight = .8, relwidth = .9, anchor = "n")

button1 = tk.Button(frame, text = "import design data", wraplength=100, bd=10, command = lambda: import_design_data())
button1.place(relheight = .20, relwidth = 1)

# entry1 = tk.Entry(frame,  font=40)
# entry1.place(relheight = .20, relwidth = .65)

button2 = tk.Button(frame, text = "import pre-post data", wraplength=150, bd=10, command = lambda: import_pre_post_data())
button2.place( rely = .20, relheight = .20, relwidth = 1)

# entry2 = tk.Entry(frame,  font=40)
# entry2.place(rely = .20, relheight = .20, relwidth = .65)

button3 = tk.Button(frame, text = "Number of Points", wraplength=50, bd=10, command = lambda: points(entry3.get()))
button3.place(relx=0.7, rely = .4, relheight = .2, relwidth = .3)

entry3 = tk.Entry(frame,  font=40)
entry3.place(rely = .4, relheight = .20, relwidth = .65)

button4 = tk.Button(frame, text = "Gap between surface", wraplength=50, bd=10, command = lambda: HEIGHT(entry4.get()))
button4.place(relx=0.7, rely = .6, relheight = .2, relwidth = .3)

entry4 = tk.Entry(frame,  font=40)
entry4.place(rely = .6, relheight = .20, relwidth = .65)

button5 = tk.Button(frame, text = "Export Area", wraplength=50, bd=10, command = lambda: write_to_csv())
button5.place(relx=0.7, rely = .8, relheight = .2, relwidth = .3)

entry5 = tk.Entry(frame,  font=40)
entry5.place(rely = .8, relheight = .20, relwidth = .65)


# lower_frame = tk.Frame(root, bg="blue", bd=10)
# lower_frame.place(relx=.5, rely=.25, relheight = 0.4, relwidth = .75, anchor = "n")


# label = tk.Label(lower_frame, text="Pre-Post Calculator", fg="#000000", font=50)
# label.place(relheight = 1, relwidth = 1)

root.mainloop()